# Fetching Pre and Post synaptic Neuron Location

## Libraries

In [2]:
! pip install standard-transform
! pip install --upgrade caveclient

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for standard-transform: filename=standard_transform-1.4.0-py3-none-any.whl size=22356 sha256=57b41cae00d939e331259faec8165b9e8fde4f1c3bcabbed977a4e8986ef0857
  Stored in directory: /home/vscode/.cache/pip/wheels/ad/3f/3f/2d6f4450cc3c31eb60e2dc6c4a24648a2e9563b610b26c9b67
Successfully built standard-transform

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for caveclient: filename=caveclient-5.14.0-py3-none-any.whl size=74844 sha256=5b5d64a7936ee0c57e683c19317b1953313f6c212965158b31f59351e799cb25
  Stored in directory: /home/vscode/.cache/pip/wheels/c5/3c/01/d965f4ed3dbeb13710b8e

In [3]:
import numpy as np
import pandas as pd
from caveclient import CAVEclient
from microns_phase3 import nda
from standard_transform import minnie_transform_vx

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


## Preparation

Function that transforms the voxel coorindate into a depth coordinate (function given to us by the authors of the Microns Project)

In [4]:
tform_vx = minnie_transform_vx()

### Querying the data from CAVEclient

In [5]:
client = CAVEclient()

In [8]:
# client.auth.save_token("64ea0271652fd86dc7fb6376cd9d52b9")

We will be using the latest public release of the dataset, i.e. version 661

In [10]:
client = CAVEclient('minnie65_public')

In [11]:
client.materialize.version = 661

In [12]:
client.materialize.get_tables()

['synapses_pni_2',
 'baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'connectivity_groups_v507',
 'proofreading_status_public_release',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v5',
 'nucleus_detection_v0',
 'aibs_soma_nuc_metamodel_preds_v117',
 'coregistration_manual_v3']

These are all the available tables, but we are interested in the *coregistration_manual_v3*, that is the one containing the information about the manually proofread neurons and in the *synapses_pni_2*, which contains all the traced synapses.

In [13]:
df = client.materialize.query_table('coregistration_manual_v3')

In [20]:
df.sample(1)

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
2059,255695,2020-09-28 22:43:14.398608+00:00,t,228.907745,89165926246694022,864691135065708612,8055,2023-04-05 22:39:27.207439+00:00,t,255695,7,5,1921,2,2.2977,12.18141,"[177584, 125968, 14894]","[nan, nan, nan]","[nan, nan, nan]"


In [25]:
prova = client.materialize.synapse_query(post_ids=864691135065708612)
prova

,id,created,superceded_id,valid,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position
0,151689578,2020-11-04 06:48:59.036865+00:00,NaN,t,15524,88745500488301082,864691136091964497,88745500488312494,864691135065708612,"[174316, 139086, 15129]","[174424, 139024, 15139]","[174370, 139046, 15134]"
1,166843041,2020-11-04 06:48:59.036865+00:00,NaN,t,15780,89729151145460606,864691136696091246,89729151145466232,864691135065708612,"[181708, 127786, 15684]","[181606, 127726, 15690]","[181670, 127770, 15683]"
2,167572754,2020-11-04 06:49:11.546047+00:00,NaN,t,200,89589994204976044,864691135684819459,89589994204970779,864691135065708612,"[180498, 139668, 15566]","[180450, 139652, 15555]","[180447, 139691, 15561]"
3,179158376,2020-11-04 06:53:25.299125+00:00,NaN,t,18860,90996269577006595,864691136008613256,90996269577003875,864691135065708612,"[190878, 131756, 15665]","[190906, 131690, 15662]","[190868, 131688, 15658]"
4,124948394,2020-11-04 06:53:27.731491+00:00,NaN,t,4284,86562488938193813,864691136339802263,86562488938201286,864691135065708612,"[158554, 127510, 15852]","[158648, 127518, 15860]","[158604, 127520, 15857]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1064,150156163,2020-11-04 09:44:36.733434+00:00,NaN,t,9260,88745088171465190,864691136816079982,88745088171473369,864691135065708612,"[174156, 136184, 15120]","[174276, 136202, 15127]","[174183, 136171, 15123]"
1065,159903802,2020-11-04 09:53:14.565729+00:00,NaN,t,14132,88885482062518104,864691135544367144,88885482062515908,864691135065708612,"[175508, 133566, 15265]","[175516, 133598, 15265]","[175512, 133532, 15261]"
1066,137305798,2020-11-04 09:53:28.828752+00:00,NaN,t,11876,87336682562698367,864691136267772055,87336682562691717,864691135065708612,"[164358, 128630, 15492]","[164272, 128552, 15483]","[164312, 128614, 15487]"
1067,142979663,2020-11-04 07:31:51.236779+00:00,NaN,t,6664,88110326364599219,864691136723374190,88110326364591933,864691135065708612,"[169788, 125130, 14943]","[169758, 125172, 14938]","[169730, 125178, 14939]"


In [14]:
df

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
0,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,864691135738685297,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]"
1,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,864691135614842827,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]"
2,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,864691135502985397,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]"
3,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,864691135387371905,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]"
4,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,864691135661410544,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13920,293075,2020-09-28 22:41:16.438872+00:00,t,287.333417,91487065153289270,864691136144304948,2323,2023-04-05 22:39:00.197594+00:00,t,293075,9,4,4693,4,3.21152,6.692787,"[194528, 118128, 23566]","[nan, nan, nan]","[nan, nan, nan]"
13921,293075,2020-09-28 22:41:16.438872+00:00,t,287.333417,91487065153289270,864691136144304948,10023,2023-04-05 22:39:40.658776+00:00,t,293075,9,4,7533,6,10.42720,-0.232141,"[194528, 118128, 23566]","[nan, nan, nan]","[nan, nan, nan]"
13922,395416,2020-09-28 22:44:46.280352+00:00,t,282.205061,97825062761246370,864691135394864117,2840,2023-04-05 22:39:00.636805+00:00,t,395416,9,3,2456,2,4.95407,3.168314,"[240528, 154032, 25770]","[nan, nan, nan]","[nan, nan, nan]"
13923,395416,2020-09-28 22:44:46.280352+00:00,t,282.205061,97825062761246370,864691135394864117,4554,2023-04-05 22:39:07.141941+00:00,t,395416,9,3,4759,4,6.51528,4.102491,"[240528, 154032, 25770]","[nan, nan, nan]","[nan, nan, nan]"


We start by setting as the index he ID associated to the root of the synapse

In [16]:
df = df.set_index('pt_root_id')

In [17]:
df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
pt_root_id,,,,,,,,,,,,,,,,,,
864691135738685297,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]"
864691135614842827,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]"
864691135502985397,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]"
864691135387371905,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]"
864691135661410544,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]"


In [18]:
df.shape

(13925, 18)

In [19]:
unit_keys_1 = df.iloc[:5000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
unit_keys_2 = df.iloc[5000:10000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
unit_keys_3 = df.iloc[10000:13925, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')

In [20]:
temp_1 = (nda.AreaMembership & unit_keys_1).fetch('brain_area')
print('1/3 Completed!')
temp_2 = (nda.AreaMembership & unit_keys_2).fetch('brain_area')
print('2/3 Completed!')
temp_3 = (nda.AreaMembership & unit_keys_3).fetch('brain_area')
print('3/3 Completed!')
area_membership = np.concatenate((temp_1, temp_2, temp_3), axis=0)
assert len(area_membership) == df.shape[0]

1/3 Completed!
2/3 Completed!
3/3 Completed!


In [21]:
df['Area Membership'] = area_membership
df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position,Area Membership
pt_root_id,,,,,,,,,,,,,,,,,,,
864691135738685297,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135614842827,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135502985397,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135387371905,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135661410544,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]",V1


In [22]:
root_ids = df.index.tolist()
len(root_ids)

13925

In [23]:
# subset_ids = root_ids[13500:13925]

In [24]:
# input_values = []
# output_values = []
# v_area = []

# for i in subset_ids:
#     input_df = client.materialize.synapse_query(post_ids=i)
    
#     input_df.rename(columns = {'pre_pt_root_id':'pt_root_id'}, inplace = True) 
   
#     df_matched = pd.merge(input_df,df,on='pt_root_id',how='inner')
#     if df_matched.empty:
#         continue
        
#     #This is the correct depth value that is retrieved by a special function created by the MICrONS team    
#     input_vertices_list = tform_vx.apply(df_matched['pt_position'])
#     graph_x_stack = np.vstack(input_vertices_list)
#     graph_x_values = graph_x_stack[:,1]

#     output_vertices = tform_vx.apply(df.at[i,'pt_position'])
#     graph_y_value = output_vertices[1]
#     if type(graph_y_value) != np.float64:
#         continue
    
#     num_points = len(graph_x_values)
#     input_values.extend(graph_x_values)
#     output_values.extend([graph_y_value]*num_points)
#     v_area.extend(df_matched['Area Membership'])

In [25]:
# df_export = pd.DataFrame(list(zip(input_values, output_values, v_area)), columns=['Inputh Depth', 'Output Depth', 'Area Membership'])
# df_export.head()

In [26]:
# df_export.to_csv('graph_values_16.csv')

In [30]:
df_list = [pd.read_csv(f"/workspaces/microns_phase3_nda/personal_notebooks/graph_values_{n}.csv") for n in range(1,17)]
complete_df = pd.concat(df_list, ignore_index=True)
complete_df.drop(['Unnamed: 0'], axis=1, inplace=True)
complete_df.head()

,Inputh Depth,Output Depth,Area Membership
0,189.550053,220.436358,RL
1,210.758237,220.436358,RL
2,171.736336,220.436358,RL
3,292.821299,220.436358,V1
4,233.734725,220.436358,V1


In [31]:
complete_df.shape

(292146, 3)

In [32]:
complete_df.to_csv('all_functional_synapses_with_area_membership.csv')